<a href="https://colab.research.google.com/github/Dinke265/NSTIP/blob/main/NSTIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rpy2

In [2]:
%load_ext rpy2.ipython


In [75]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [76]:
%cd /content/drive/My Drive/NSTIP/

/content/drive/My Drive/NSTIP


In [ ]:
%%R

install.packages("tidyverse")
install.packages("dplyr")
install.packages("readr")

library(tidyverse)
library(dplyr)
library(readr)

In [77]:
%%R
allgradsdata <- read_csv("/content/drive/My Drive/NSTIP/AllGraduatesData2022.csv")
stemgrads <- read_csv("/content/drive/My Drive/NSTIP/STEMgraduates.csv")
head(allgradsdata)

Rows: 992 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): PUBLIC INSTITUTION, PRIVATE INSTITUTUIN, PROGRAMS, LEVEL
dbl (6): NO.OF FEMALES, NO.OF MALES, STEM, STI, ICT, MISCELAANEOUSS

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



Error: '/content/drive/My Drive/NSTIP/STEMgraduates.csv' does not exist.


In [73]:
%%R
head(stemgrads)

names(stemgrads) <- tolower(names(stemgrads))

print(names(stemgrads))

 [1] "public institution"     "private institutuin"    "stem program"          
 [4] "sti program"            "ict program"            "miscellaneous programs"
 [7] "no.of females"          "no.of males"            "level"                 
[10] "...10"                  "...11"                  "...12"                 
[13] "...13"                  "...14"                  "...15"                 
[16] "...16"                  "...17"                  "...18"                 
[19] "...19"                  "...20"                  "...21"                 
[22] "...22"                  "...23"                  "...24"                 
[25] "...25"                  "...26"                  "...27"                 


In [65]:
%%R
# Print the unique categories for the level
print(unique(stemgrads$level))


 [1] "Bacherors"                  "Diploma"                   
 [3] "PhD"                        "Masters"                   
 [5] "Degree"                     "Certificate"               
 [7] "Bacheror"                   "Dipoma"                    
 [9] NA                           "Bachelor's degree"         
[11] "Undergraduate"              "PHD"                       
[13] "Postgraduate Diploma Level" "diploma"                   
[15] "CERTIFICATE"                "Bachelors"                 
[17] "Doctors"                    "Diplomas"                  
[19] "UNDERGRADUATE"              "7"                         
[21] "6"                          "4"                         
[23] "5"                          "Post Graduate"             
[25] "BACHELOR'S"                 "DIPLOMA"                   
[27] "MASTER'S"                   "DEGREE"                    
[29] "CA Level 1"                 "Foundetion Diploma"        
[31] "Diploma Level 5"            "Diploma Level"      

In [68]:
%%R
#Clean the level variable

stemgrads <- stemgrads %>%
  mutate(level = case_when(
    level == "Degree" ~ "Bachelors",
    level == "Degree Level" ~ "Bachelors",
    level == "Undergraduate" ~ "Bachelors",
    level == "BACHELOR'S" ~ "Bachelors",
    level == "DEGREE" ~ "Bachelors",
    level == "Bachelor's degree" ~ "Bachelors",
    level == "Bacherors" ~ "Bachelors",
    level == "Bacheror" ~ "Bachelors",
    level == "Bacheros" ~ "Bachelors",
    level == "Masters Level" ~ "Masters",
    level == "MASTERS DEGREE" ~ "Masters",
    level == "UNDERGRADUATE" ~ "Bachelors",
    level == "MASTER'S" ~ "Masters",
    level == "Diploma Level" ~ "Diploma",
    level == "Foundetion Diploma" ~ "Diploma",
    level == "Diploma Level 5" ~ "Diploma",
    level == "NCC LEVEL 5 DIPLOMA" ~ "Diploma",
    level == "NCC LEVEL 4 DIPLOMA" ~ "Diploma",
    level == "diploma" ~ "Diploma",
    level == "DIPLOMA" ~ "Diploma",
    level == "Level 4" ~ "Diploma",
    level == "Dipoma" ~ "Diploma",
    level == "Level 6" ~ "Bachelors",
    level == "Chichewa Program" ~ "Certificate",
    level == "English Program" ~ "Certificate",
    level == "CERTIFICATE" ~ "Certificate",
    level == "PhD" ~ "Doctorate",
    level == "Doctors" ~ "Doctorate",
    level == "7" ~ "Bachelors",
    level == "9" ~ "Masters",
    level == "10" ~ "Doctorate",
    level == "PHD" ~ "Doctorate",
    level == "4" ~ "Certificate",
    level == "5" ~ "Diploma",
    level == "6" ~ "Diploma",
    level == "Post Graduate" ~ "Masters",
    level == "Postgraduate" ~ "Masters",
    level == "CA Level 1" ~ "Diploma",
    level == "Postgraduate Diploma Level" ~ "Postgraduate Other",
    TRUE ~ as.character(level)
  ))


In [69]:
%%R
print(unique(stemgrads$level)) # Check the categories for the level have been correctly replaced

[1] "Bachelors"          "Diploma"            "Doctorate"         
[4] "Masters"            "Certificate"        NA                  
[7] "Postgraduate Other" "Diplomas"          


In [70]:
%%R
#Renaming some variables
stemgrads <- stemgrads%>%
  rename(
    males = `no.of males`,
    females = `no.of females`,
    private = `private institutuin`,
    public = `public institution`)

print(names(stemgrads))

 [1] "public"                 "private"                "stem program"          
 [4] "sti program"            "ict program"            "miscellaneous programs"
 [7] "females"                "males"                  "level"                 
[10] "...10"                  "...11"                  "...12"                 
[13] "...13"                  "...14"                  "...15"                 
[16] "...16"                  "...17"                  "...18"                 
[19] "...19"                  "...20"                  "...21"                 
[22] "...22"                  "...23"                  "...24"                 
[25] "...25"                  "...26"                  "...27"                 


Creating an indicator variable for institution type.
1= public institution
0= private institution

In [40]:
%%R

allgradsdata <- allgradsdata %>%
  mutate(row_id = row_number()) %>%
  pivot_longer(cols = c(private, public), names_to = "type_institution", values_to = "institution_value") %>%
  filter(!is.na(institution_value)) %>%
  mutate(institution_type = ifelse(institution_type == "public", 1, 0)) %>%
  select(-row_id)

In [71]:
%%R
#stemgrads <- stemgrads[order(stemgrads$institution_value, stemgrads$program), ]
write.csv(stemgrads, "stemgradsdata.csv", row.names = TRUE)


Numbers of males and females by degree level

In [ ]:
%%R
allgradsdata %>%
  group_by(level) %>%
  summarise(
    total_males = sum(males, na.rm = TRUE),
    total_females = sum(females, na.rm = TRUE)
  ) %>%
  print()


# **Read STEM, STI and ICT data**

In [55]:
%%R
stemgrads <- read_csv("/content/drive/My Drive/NSTIP/STISTEMgraduates.csv")

New names:
• `` -> `...10`
• `` -> `...11`
• `` -> `...12`
• `` -> `...13`
• `` -> `...14`
• `` -> `...15`
• `` -> `...16`
• `` -> `...17`
• `` -> `...18`
• `` -> `...19`
• `` -> `...20`
• `` -> `...21`
• `` -> `...22`
• `` -> `...23`
• `` -> `...24`
• `` -> `...25`
• `` -> `...26`
• `` -> `...27`
Rows: 1196 Columns: 27
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): PUBLIC INSTITUTION, PRIVATE INSTITUTUIN, STEM PROGRAM, STI PROGRAM...
dbl  (2): NO.OF FEMALES, NO.OF MALES
lgl (17): ...11, ...12, ...13, ...14, ...15, ...16, ...17, ...18, ...19, ......

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
